In [ ]:
import os
import fastavro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from correction.compute import *

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
data = pd.read_csv("../data_examples/csv/raw_detections.csv", index_col=False)

In [ ]:
import os

In [ ]:
data = pd.read_parquet("./det_sample.parquet")

In [ ]:
data.head()

In [ ]:
# test c2
data.at[180, "isdiffpos"] = 'f'

In [ ]:
# test c3
data.at[150, "distnr"] = 0.1

In [ ]:
print(data.objectId.unique(), data.fid.unique())

In [ ]:
pd.options.display.max_rows = 999
display(data[["objectId", "fid", "magpsf", "distnr", "chinr", "sharpnr"]])

In [ ]:
def apply_correction_df(data):
    
    # apply correction where possible, save corrected column
    df = data.copy()
    #display(df.head(1))
    #print(df.objectId.unique(), df.fid.unique())
    df['isdiffpos'] = df['isdiffpos'].map({'t': 1., 'f': -1.})
    df["corr"] = df["distnr"] < 1.4
    correction_results = df.apply(lambda x: correction(x.magnr, x.magpsf, x.sigmagnr, x.sigmapsf, x.isdiffpos) if x["corr"] else (np.nan, np.nan, np.nan), axis=1, result_type="expand")
    df["magpsf_corr"], df["sigmapsf_corr"], df["sigmapsf_corr_ext"] = correction_results[0], correction_results[1], correction_results[2]
    
    # check for consistency
    idxmin = df.candid.idxmin()
    df["corr_magstats"] = df.loc[idxmin]["corr"]
    
    # check if suspicious behavior in the light curve (c1 | c2 | c3)
    mask = ((df["corr"] == False) & (df.isdiffpos == -1)) | (df.corr_magstats & (df["corr"] == False)) | ((df.corr_magstats == False) & df["corr"])
    df["dubious"] = mask
    
    # stellar_object
    idxmin = df.candid.idxmin()
    nearZTF = (df.loc[idxmin].distnr < 1.4) # near a ZTF object
    nearPS1 = (df.loc[idxmin].distpsnr1 < 1.4) # near a PS1 object
    stellarPS1 = (df.loc[idxmin].sgscore1 > 0.4)  # nearest object in PS1 is stelar
    stellarZTF = (df.loc[idxmin].chinr < 2) & (df.loc[idxmin].sharpnr > -0.13) & (df.loc[idxmin].sharpnr < 0.1) # nearest object in ZT is stellar
    df["stellar_object"] = (nearZTF & nearPS1 & stellarPS1) | (nearZTF & ~nearPS1 & stellarZTF)
    df["stellar_magstats"] = (nearZTF & stellarZTF)
    
    return df

In [ ]:
def apply_magstats(df):
    response = {}
    idxmin = df.candid.idxmin()
    idxmax = df.candid.idxmax()
    nearZTF = (df.loc[idxmin].distnr < 1.4) # near a ZTF object
    nearPS1 = (df.loc[idxmin].distpsnr1 < 1.4) # near a PS1 object
    stellarPS1 = (df.loc[idxmin].sgscore1 > 0.4)  # nearest object in PS1 is stelar
    stellarZTF = (df.loc[idxmin].chinr < 2) & (df.loc[idxmin].sharpnr > -0.13) & (df.loc[idxmin].sharpnr < 0.1) # nearest object in ZT is stellar 
    response["ndet"] = len(df)
    response["rfid"] = df.loc[idxmin].rfid # ?
    response["nrfid"] = len(df.rfid.dropna().unique())
    response["magnr"] = df.loc[idxmin].magnr # ?
    response["stellar_object"] = (nearZTF & nearPS1 & stellarPS1) | (nearZTF & ~nearPS1 & stellarZTF)
    response["stellar_magstats"] = (nearZTF & stellarZTF)
    response["magap_mean"] = df.magap.mean()
    response["magap_median"] = df.magap.median()
    response["magap_max"] = df.magap.max()
    response["magap_min"] = df.magap.min()
    response["magap_fisrt"] = df.loc[idxmin].magap
    response["magap_last"] = df.loc[idxmax].magap
    response["first_mjd"] = df.loc[idxmin].jd - 2400000.5
    response["last_mjd"] = df.loc[idxmax].jd - 2400000.5
    return pd.Series(response)

In [ ]:
dflarge = data.groupby(["objectId", "fid"]).apply(apply_correction_df)

In [ ]:
dflarge[["objectId", "fid", "distnr", "distpsnr1", "sgscore1", "chinr", "sharpnr", "corr", "corr_magstats", "dubious", "stellar_object", "stellar_magstats"]]

In [ ]:
groups = data.groupby(["objectId", "fid"])

In [ ]:
pd.options.display.max_rows = 999
display(data[["objectId", "fid", "magpsf", "distnr", "chinr", "sharpnr"]])

In [ ]:
def apply_correction_df(data):
    
    # apply correction where possible, save corrected column
    df = data.copy()
    #display(df.head(1))
    #print(df.objectId.unique(), df.fid.unique())
    df['isdiffpos'] = df['isdiffpos'].map({'t': 1., 'f': -1.})
    df["corr"] = df["distnr"] < 1.4
    correction_results = df.apply(lambda x: correction(x.magnr, x.magpsf, x.sigmagnr, x.sigmapsf, x.isdiffpos) if x["corr"] else (np.nan, np.nan, np.nan), axis=1, result_type="expand")
    df["magpsf_corr"], df["sigmapsf_corr"], df["sigmapsf_corr_ext"] = correction_results[0], correction_results[1], correction_results[2]
    
    # check for consistency
    idxmin = df.candid.idxmin()
    df["corr_magstats"] = df.loc[idxmin]["corr"]
    
    # check if suspicious behavior in the light curve (c1 | c2 | c3)
    mask = ((df["corr"] == False) & (df.isdiffpos == -1)) | (df.corr_magstats & (df["corr"] == False)) | ((df.corr_magstats == False) & df["corr"])
    df["dubious"] = mask
    
    # stellar_object
    idxmin = df.candid.idxmin()
    nearZTF = (df.loc[idxmin].distnr >= 0) & (df.loc[idxmin].distnr < 1.4) # near a ZTF object
    nearPS1 = (df.loc[idxmin].distpsnr1 < 1.4) # near a PS1 object
    stellarPS1 = (df.loc[idxmin].sgscore1 > 0.4)  # nearest object in PS1 is stelar
    stellarZTF = (df.loc[idxmin].chinr < 2) & (df.loc[idxmin].sharpnr > -0.13) & (df.loc[idxmin].sharpnr < 0.1) # nearest object in ZT is stellar
    df["stellar_object"] = (nearZTF & nearPS1 & stellarPS1) | (nearZTF & ~nearPS1 & stellarZTF)
    df["stellar_magstats"] = (nearZTF & stellarZTF)
    
    return df

In [ ]:
data.loc[data.objectId == "ZTF18aaxnouf"][["distnr", "magnr"]]

In [ ]:
dflarge = data.loc[data.objectId == "ZTF18aaxnouf"].groupby(["objectId", "fid"]).apply(apply_correction_df)

In [ ]:
dflarge["mjd"] = dflarge.jd - 2400000.5

In [ ]:
dflarge[["objectId", "rfid", "fid", "candid", "mjd", "magpsf", "magnr", "isdiffpos", "magpsf_corr", "distnr", "distpsnr1", "sgscore1", "chinr", "sharpnr", "corr", "corr_magstats", "dubious", "stellar_object", "stellar_magstats"]]

In [ ]:
colors = {1: 'g', 2: 'r'}
period = {}
period['ZTF19aazzpje'] = None #'ZTF20aaelulu' #'ZTF19aazzpje' # SN
period['ZTF18aaiscil'] = None # weird
period['ZTF18abdgukn'] = None # AGN
period['ZTF18aazxcwf'] = 0.614515 # RRL
period['ZTF17aaaedmd'] = None # LPV
period["ZTF19abaejrh"] = None
period["ZTF20aaelulu"] = None

for oid in dflarge.objectId.unique():
    fig, ax = plt.subplots(ncols = 3, sharey = False, figsize=(30, 6))
    maskoid = dflarge.objectId == oid
    for fid in dflarge.loc[maskoid].fid.unique():
        mask = maskoid & (dflarge.fid == fid)
        for pos in dflarge.loc[mask].isdiffpos.unique():
            
            mask = mask & (dflarge.isdiffpos == pos)
            mask_diff = mask & (dflarge.sigmapsf < 2)
            mask_corr = mask & (dflarge.sigmapsf_corr < 2)
            mask_corr_ext = mask & (dflarge.sigmapsf_corr_ext < 2)
        
            color = colors[fid]
            marker = 'o' if pos == 't' else '*'

            if period[oid] is None:
                ax[0].errorbar(dflarge.loc[mask_diff].jd, dflarge.loc[mask_diff].magpsf, yerr=dflarge.loc[mask_diff].sigmapsf, c=color, marker=marker, lw=0, elinewidth=1, alpha=1, label="magpsf")
                ax[1].errorbar(dflarge.loc[mask_corr].jd, dflarge.loc[mask_corr].magpsf_corr, yerr=dflarge.loc[mask_corr].sigmapsf_corr, c=color, marker=marker, lw=0, elinewidth=1, alpha=1, label="magpsf_corr")
                ax[2].errorbar(dflarge.loc[mask_corr_ext].jd, dflarge.loc[mask_corr_ext].magpsf_corr, yerr=dflarge.loc[mask_corr_ext].sigmapsf_corr_ext, c=color, marker=marker, lw=0, elinewidth=1, alpha=1, label="magpsf_corr_ext")
            else:
                ax[0].errorbar(np.mod(dflarge.loc[mask_diff].jd, period[oid]), dflarge.loc[mask_diff].magpsf, yerr=dflarge.loc[mask_diff].sigmapsf, c=color, marker=marker, lw=0, elinewidth=1, alpha=1, label="magpsf")
                ax[1].errorbar(np.mod(dflarge.loc[mask_corr].jd, period[oid]), dflarge.loc[mask_corr].magpsf_corr, yerr=dflarge.loc[mask_corr].sigmapsf_corr, c=color, marker=marker, lw=0, elinewidth=1, alpha=1, label="magpsf_corr")
                ax[2].errorbar(np.mod(dflarge.loc[mask_corr_ext].jd, period[oid]), dflarge.loc[mask_corr_ext].magpsf_corr, yerr=dflarge.loc[mask_corr_ext].sigmapsf_corr_ext, c=color, marker=marker, lw=0, elinewidth=1, alpha=1, label="magpsf_corr_ext")
    
    for i in range(3):
        ax[i].set_title(oid)
        ax[i].set_ylim(ax[i].get_ylim()[::-1])
        ax[i].legend()
        if period[oid] is None:
            ax[i].set_xlabel("jd")
        else:
            ax[i].set_xlabel("phase")